In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
%pip install git+https://github.com/amazon-science/chronos-forecasting.git

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-eoit7cky
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-eoit7cky
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit eb7bdfc047de3e7af972b4ee7cf23a7968b7daa3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12

In [3]:
%pip install gluonts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.9 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import torch
from gluonts.dataset.pandas import PandasDataset
from chronos import ChronosPipeline

In [5]:
# Load data

from google.colab import files
uploaded = files.upload()

Saving test_tscv.csv to test_tscv.csv
Saving train_tscv.csv to train_tscv.csv


In [7]:
# read data

train_tscv = pd.read_csv("train_tscv.csv")
test_tscv = pd.read_csv("test_tscv.csv")

# make the yearmonth column as date format

train_tscv['yearmonth'] = pd.to_datetime(train_tscv['yearmonth']) + pd.offsets.MonthEnd(0)
train_tscv = train_tscv.sort_values(by = 'yearmonth')
train_tscv = train_tscv.set_index('yearmonth')

test_tscv['yearmonth'] = pd.to_datetime(test_tscv['yearmonth']) + pd.offsets.MonthEnd(0)
test_tscv = test_tscv.sort_values(by = 'yearmonth')
test_tscv = test_tscv.set_index('yearmonth')

train_tscv.head()

,id,unique_id,horizon,stock_distributed
yearmonth,,,,
2016-01-31,C1014AS27137,C1014AS27137-1,1,0
2016-01-31,C4022AS27137,C4022AS27137-1,1,0
2016-01-31,C1098AS27000,C1098AS27000-1,1,20
2016-01-31,C2004AS27132,C2004AS27132-1,1,0
2016-01-31,C2064AS27137,C2064AS27137-2,2,40


In [8]:
# create test and train dataset

series_train = train_tscv[['unique_id', 'stock_distributed']]
series_test = test_tscv[['unique_id', 'stock_distributed']]

In [10]:
# initialize CHRONOS pipeline

pipeline = ChronosPipeline.from_pretrained("amazon/chronos-t5-base",
                                           device_map="cuda",
                                           torch_dtype=torch.bfloat16)

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/806M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [11]:
# Get the unique IDs
unique_ids = series_train['unique_id'].unique()

# Create a dictionary to store predictions for each id
predictions = {}

for id_value in unique_ids:
    # Filter the DataFrame for the current id
    series_train_filter = series_train[series_train['unique_id'] == id_value]

    # Ensure the 'stock_distributed' column is converted to a tensor
    stock_distributed_tensor = torch.tensor(series_train_filter['stock_distributed'].values)

    # Execute the prediction
    chronos_pred = pipeline.predict(
        context=stock_distributed_tensor,
        prediction_length=3,
        num_samples=1000,
    )

    # Store the prediction in the dictionary with the id as the key
    predictions[id_value] = chronos_pred

In [12]:
# List to hold the structured data for the DataFrame
data = []

# Iterate over the predictions dictionary
for id_value, tensor in predictions.items():
    # Convert tensor to numpy array
    array = tensor.numpy()

    # Iterate over each prediction step (axis=1)
    for step in range(array.shape[1]):
        # Create a row dictionary with the id and prediction_step
        row = {
            'id': id_value,
            'prediction_step': f'X{step}'
        }

        # Add the prediction values for this step as columns X0, X1, ..., X999
        for sample in range(array.shape[2]):
            row[f'X{sample}'] = array[0, step, sample]

        # Append the row to the data list
        data.append(row)

# Convert list of dictionaries to a DataFrame
df_predictions = pd.DataFrame(data)


In [13]:
# prepare final df

df_predictions_long = pd.melt(df_predictions, id_vars=['id', 'prediction_step'], var_name='fc_step', value_name='value')
chronos_prob = df_predictions_long.pivot(index=['id', 'fc_step'], columns='prediction_step', values='value').reset_index()
#chronos_prob = chronos_prob.drop('prediction_step', axis = 1)
chronos_prob['model'] = 'chronos'


In [14]:
chronos_prob

prediction_step,id,fc_step,X0,X1,X10,X100,X101,X102,X103,X104,...,X991,X992,X993,X994,X995,X996,X997,X998,X999,model
0,C1004AS27000-1,X0,1.005695e+01,2.202686e-07,1.999829e+01,7.051422e+00,2.103867e+01,2.600934e+01,2.196345e+01,2.196345e+01,...,1.005695e+01,1.999829e+01,3.803144e+01,1.595240e+01,3.005524e+01,7.051422e+00,4.589204e+01,7.051422e+00,2.202686e-07,chronos
1,C1004AS27000-1,X1,1.005695e+01,2.202686e-07,1.294687e+01,1.803314e+01,5.190309e+01,5.201868e+01,3.202039e+01,1.595240e+01,...,2.600934e+01,2.202686e-07,2.600934e+01,9.016572e+00,5.606458e+01,2.202686e-07,2.202686e-07,1.294687e+01,2.202686e-07,chronos
2,C1004AS27000-1,X2,7.051422e+00,2.202686e-07,2.202686e-07,1.595240e+01,1.294687e+01,1.999829e+01,5.005354e+01,1.595240e+01,...,2.202686e-07,2.300382e+01,1.595240e+01,4.970675e+00,5.999488e+01,2.202686e-07,2.202686e-07,7.051422e+00,2.202686e-07,chronos
3,C1004AS27000-2,X0,1.904342e+01,3.397944e+01,4.194531e+01,2.800503e+01,3.397944e+01,6.011747e+01,4.194531e+01,2.999650e+01,...,7.206628e+01,7.629815e+01,3.198797e+01,6.410040e+01,6.808334e+01,6.808334e+01,3.597091e+01,9.957345e+00,4.406125e+01,chronos
4,C1004AS27000-2,X1,6.795888e+01,2.999650e+01,6.011747e+01,1.804769e+01,5.999300e+01,4.804419e+01,2.371698e-07,1.904342e+01,...,2.402209e+01,1.605622e+01,4.194531e+01,2.999650e+01,2.601356e+01,3.995384e+01,4.978673e+00,9.957345e+00,1.605622e+01,chronos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6790,C5066AS27138-2,X1,2.795455e+01,3.005865e+00,2.603470e-08,4.003266e+00,2.795455e+01,5.000667e+00,2.603470e-08,2.603470e-08,...,2.603470e-08,3.005865e+00,2.603470e-08,4.003266e+00,6.011730e+00,2.603470e-08,2.603470e-08,4.003266e+00,4.003266e+00,chronos
6791,C5066AS27138-2,X2,2.795455e+01,2.603470e-08,2.603470e-08,2.603470e-08,2.795455e+01,4.003266e+00,5.998067e+00,2.603470e-08,...,2.603470e-08,5.000667e+00,2.603470e-08,2.603470e-08,2.603470e-08,2.603470e-08,2.603470e-08,2.603470e-08,2.603470e-08,chronos
6792,C5066AS27138-3,X0,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,...,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.003910e+00,1.198338e+01,2.545615e-08,chronos
6793,C5066AS27138-3,X1,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,...,2.001238e+01,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.545615e-08,2.003910e+00,7.000326e+00,2.545615e-08,chronos


In [15]:
# save file

chronos_prob.to_csv('chronos_pred_cv.csv', index = False)
files.download("chronos_pred_cv.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>